In [1]:
from collections import defaultdict
from itertools import combinations_with_replacement, permutations
import pandas as pd
from IPython.display import display, clear_output

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [7]:
class Intcomp:
    def __init__(self, data):
        self.data = defaultdict(lambda: 0)
        for i, d in enumerate(data):
            self.data[i] = d

        self.data_original = defaultdict(lambda: 0)
        for i, d in enumerate(data):
            self.data_original[i] = d

        self.i = 0
        self.relative_base = 0
        self.screen = pd.DataFrame()
        self.ball_pos = 0
        self.paddle_pos = 0
        self.input_count = 0

    def process(self, reset=False):
        output = self.__process2()
        if reset:
            self.data = self.data_original.copy()
            self.i = 0
            self.relative_base = 0
        return output

    def mode0(self, idx):
        return self.data[self.data[idx]]

    def mode1(self, idx):
        return self.data[idx]

    def mode2(self, idx):
        return self.data[self.data[idx]+self.relative_base]

    def code01(self, modes):
        if modes[0] == '2':
            self.data[self.data[self.i+3]+self.relative_base] = eval(f'self.mode{modes[2]}({self.i+1})') + eval(f'self.mode{modes[1]}({self.i+2})')
        else:
            self.data[self.data[self.i+3]] = eval(f'self.mode{modes[2]}({self.i+1})') + eval(f'self.mode{modes[1]}({self.i+2})')

    def code02(self, modes):
        if modes[0] == '2':
            self.data[self.data[self.i+3]+self.relative_base] = eval(f'self.mode{modes[2]}({self.i+1})') * eval(f'self.mode{modes[1]}({self.i+2})')
        else:
            self.data[self.data[self.i+3]] = eval(f'self.mode{modes[2]}({self.i+1})') * eval(f'self.mode{modes[1]}({self.i+2})')

    def code03(self, input_, mode='0'):
        if mode=='2':
            self.data[self.data[self.i+1]+self.relative_base] = input_
        else:
            self.data[self.data[self.i+1]] = input_

    def code04(self, mode):
        self.i += 2
        # print(eval(f'self.mode{mode}({self.i-1})'))
        return eval(f'self.mode{mode}({self.i-1})')

    def code07(self, modes):
        if eval(f'self.mode{modes[2]}({self.i+1})') < eval(f'self.mode{modes[1]}({self.i+2})'):
            self.data[self.data[self.i+3] + (self.relative_base if modes[0]=='2' else 0)] = 1
        else:
            self.data[self.data[self.i+3] + (self.relative_base if modes[0]=='2' else 0)] = 0

    def code08(self, modes):
        if eval(f'self.mode{modes[2]}({self.i+1})') == eval(f'self.mode{modes[1]}({self.i+2})'):
            self.data[self.data[self.i+3] + (self.relative_base if modes[0]=='2' else 0)] = 1
        else:
            self.data[self.data[self.i+3] + (self.relative_base if modes[0]=='2' else 0)] = 0

    def code09(self, mode):
        self.relative_base += eval(f'self.mode{mode}({self.i+1})')

    def __process2(self):
        while True:
            code = str(self.data[self.i])[-2:].zfill(2)
            self.last_code = code
            # print(code)
            if code in ('01', '02', '07', '08'):
                modes = str(self.data[self.i])[:-2].zfill(3)
                eval(f'self.code{code}(modes)')
                self.i += 4
            elif code == '03':
                mode = str(self.data[self.i])[:-2].zfill(1)
                clear_output()
                display(self.screen.replace({0:'.', 2: '#', 3: '–', 4: 'O'}))
                button_press = input()
                mapping = {'': 0, 'a': -1, 'd':1}
                self.code03(mapping[button_press], mode)
                if self.ball_pos > self.paddle_pos:
                    joystick = 1
                elif self.ball_pos < self.paddle_pos:
                    joystick = -1
                else:
                    joystick = 0
                self.input_count += 1
                self.code03(joystick, mode)
                self.i += 2
            elif code == '04':
                mode = str(self.data[self.i])[:-2].zfill(1)
                return self.code04(mode)
            elif code == '05':
                modes = str(self.data[self.i])[:-2].zfill(2)
                if eval(f'self.mode{modes[1]}({self.i+1})') != 0:
                    self.i = eval(f'self.mode{modes[0]}({self.i+2})')
                else:
                    self.i += 3
            elif code == '06':
                modes = str(self.data[self.i])[:-2].zfill(2)
                if eval(f'self.mode{modes[1]}({self.i+1})') == 0:
                    self.i = eval(f'self.mode{modes[0]}({self.i+2})')
                else:
                    self.i += 3
            elif code == '09':
                mode = str(self.data[self.i])[:-2].zfill(1)
                self.code09(mode) 
                self.i += 2
            elif code == '99':
                # self.i += 1
                break
            else:
                print('ERROR')
                break

In [4]:
with open('day13data.txt') as f:
    data = list(map(int, f.read().split(',')))
data[0] = 2

In [ ]:
last_out = 0
draw = []
screen = pd.DataFrame()
comp = Intcomp(data)
while True:
    last_out = comp.process()
    if last_out is None:
        break
    draw.append(last_out)
    if not len(draw) % 3:
        comp.screen.loc[draw[-2], draw[-3]] = draw[-1]
        if draw[-1] == 4:
            comp.ball_pos = draw[-3]
        if draw[-1] == 3:
            comp.paddle_pos = draw[-3]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,-1
0,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,918.0
1,1.0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1.0,NaN
2,1.0,.,#,.,.,#,.,#,#,.,#,.,#,#,#,#,#,#,.,.,#,.,#,#,#,#,.,#,.,.,.,#,.,.,#,.,.,.,.,.,.,.,#,.,.,1.0,NaN
3,1.0,.,.,#,.,.,#,#,.,#,.,.,.,.,#,#,#,.,.,.,.,.,.,#,#,#,.,.,#,.,.,#,#,.,#,#,.,.,.,.,#,#,.,.,.,1.0,NaN
4,1.0,.,.,.,.,.,.,.,.,.,.,.,#,#,#,#,.,#,#,.,.,#,.,.,#,.,.,.,.,#,.,.,#,#,#,.,.,#,#,.,#,#,#,#,.,1.0,NaN
5,1.0,.,#,.,.,#,#,#,#,#,.,.,#,.,.,.,#,.,.,#,.,.,.,.,.,.,.,.,.,#,.,.,#,.,#,.,.,#,.,.,.,.,.,.,.,1.0,NaN
6,1.0,.,#,#,.,#,#,.,#,#,.,.,#,#,.,.,.,#,.,.,#,.,.,.,#,#,#,#,#,#,.,#,.,.,.,.,.,#,.,#,.,#,#,.,.,1.0,NaN
7,1.0,.,#,.,#,.,#,.,#,.,.,#,.,.,.,#,#,.,.,.,.,.,#,.,.,.,.,#,.,.,.,.,.,#,.,.,.,#,.,#,#,.,#,#,.,1.0,NaN
8,1.0,.,#,.,#,.,.,#,.,.,#,.,.,#,#,.,.,#,#,#,.,.,#,.,.,#,#,.,.,#,.,#,.,.,.,.,.,#,#,#,.,.,#,#,.,1.0,NaN
9,1.0,.,#,.,.,.,#,#,#,.,#,.,.,#,.,.,.,.,#,#,#,.,.,#,.,#,.,#,#,#,#,.,.,#,.,.,.,.,#,.,.,#,#,.,.,1.0,NaN


In [6]:
comp.input_count

5130